In [ ]:
##imports from libraries
import pandas as pd
import numpy as np
import time
import math
import sys
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from IPython.core.debugger import set_trace


# the "resource" library is not available on windows. 
# if it can be imported, we can use it! For example, while running on google colab
try:
    import resource
    print("Succesfully imported 'resource' package")
except:
    print("Failed on importing 'resource' package")

### Load the data:
We first load the data into ghgdata. We will only use 2000 data points in this implementation, so its enough to use 7 files.

In [ ]:
N_sites = 7
N_cols = 327
dghg = 15
Nghg = N_sites * N_cols
ghgdata = np.zeros((dghg+1, Nghg))

pathprefix = "./Data/ghg_data/ghg_data/ghg.gid.site"
for i in range(N_sites):
    filename = pathprefix + str(i + 1).zfill(4) + ".dat"
    subdata = np.genfromtxt(filename, delimiter=" ")
    ghgdata[:,i*N_cols:(i+1)*N_cols] = subdata

ghgdata = ghgdata.T

### Define function to find the smoothness $L$ of the cost function, given a dataset:
We have that for $L$ to guarantee L-smoothness of $f(w)$, it must fulfill the following inequality:
$$ L \succcurlyeq \frac{1}{4N} \sum_i y_i^2 x_i x_i^T + 2\lambda  $$
$$  = \frac{1}{4N} Z^T Z +2\lambda, $$
$$ Z = \begin{bmatrix}y_1 x_1^T \\ \vdots\\ y_N x_N^T\end{bmatrix}.$$

Which is equivalent to $L \geq \frac{1}{4N}\lambda_{\max}(Z^T Z) + 2\lambda $, where $\lambda_{\max}(Z^T Z)$ is the largest eigenvalue of $Z^T Z$.

We implement the finding of the smallest $L$ in the function ``find_smoothness()``.

In [ ]:
def find_smoothness(X, Y, lambda_):
    # finds smallest L to guarantee L-smoothness of the Logistic Ridge Regression, given data X, Y
    # we have found it analytically as the largest eigenvalue of (1/(4N)) * Z.T @ Z + 2 * lambda.
    
    N, d = X.shape
    assert Y.shape == (N,1)
    
    YX = Y * X
    Z = YX.T @ YX
    eigs, _ = np.linalg.eig((1/(4*N))*Z)
    L = np.max(eigs) + 2*lambda_
    
    return L

### Preprocessing of the response variable
We use the first 15 variables of the data as inputs $x_i$, and the 16th row as output.
We found that using the raw output data came with some numerical issues, due to the products $y_i x_i$ being very large quite often.
We therefore threshold the output data into a binary variable in $\{-1,+1\}$.

We here load the ``ghgdata`` into ``X`` and ``Y`` matrices, and perform the thresholding on the ``Y`` matrix.

The plots show histograms of ``Y`` before and after thresholding.

In [ ]:
X = ghgdata[:, 0:dghg]
Y = ghgdata[:, [dghg]]
plt.figure(1)
_ = plt.hist(Y, bins=100)

# threshold Y to binary
threshold = 30
mask = Y < threshold
Y[mask] = -1
Y[~mask] = 1
plt.figure(2)
_ = plt.hist(Y)

### Split data into training and testing datasets
We are not really interested in finding a good classifier of the data, but only to verify our implementation of GD, SGD, SVRG and SAG.
Therefore we limit the simulations to only 1000 datapoints for training, to decrease the run time.

In [ ]:
# Split train and test data here: (X_train, Y_train, X_test, Y_test)

N_train = math.floor(Nghg*0.75)
N_test = math.floor(Nghg*0.25) + 1

N_train = 1000
N_test = 1000

# train_idx = np.random.choice(Nghg, size=N_train, replace=False)
train_idx = np.arange(N_train)
X_train = X[train_idx, :]
Y_train = Y[train_idx]

# test_idx_bool = ~np.isin(np.arange(Nghg), train_idx)
test_idx = np.arange(N_train, N_train + N_test)
X_test = X[test_idx, :]
Y_test = Y[test_idx]

## Implementing Cost and Gradient functions
#### The logistic ridge regression
$$ f(x) = \frac{1}{N}\sum_{i\in[N]} f_i(x) + \lambda\lVert w \rVert_2^2 $$
has the gradient: 
$$ \nabla f(x) = \frac{1}{N}\sum_{i\in[N]} \nabla f_i(x) + 2\lambda w, $$
$$ \nabla f_i(x) = -y_i x_i \frac{e^{-y_ix_i^T w}}{1 + e^{-y_ix_i^T w}} = -y_i x_i \frac{1}{1 + e^{y_i x_i^T w}}.$$
We use the latter form for computing gradients, since it has better numerical stability (the exponent can take "inf" values, thus the former form is undefined, while the latter is identically zero).

We define the gradient estimate for data point $i$ including regularization as:
$$  \hat{g}_i(w) = \nabla f_i(w) + 2\lambda w.$$

The ``cost()`` function is an implementation of $f(w)$.
The ``function_gradient()`` is an implementation of $\nabla f(w)$, given ``X, Y`` and ``w``.

In [ ]:
def cost(x, y, w, lambda_ = 0.01):
    N, d = x.shape
    value = np.sum(np.log(1 + np.exp(- y*x @ w)))
    norm_w = np.linalg.norm(w)
    c = lambda_ * norm_w ** 2
    return value/N + c 

def function_gradient(X, Y, w, lambda_):
    N, d = X.shape
    assert Y.shape == (N,1)
    assert w.shape == (d,1)
    output = np.zeros((N,1))
    YX = Y * X # (N,d)
    YXw = YX @ w # (N,1)
    exp_vec = 1/(1 + np.exp(YXw)) # (N,1)
    grad_array = -YX * exp_vec # (N,d)
    output = np.sum(grad_array, axis=0) # (1,d)
    output = (1/N) * output.reshape(d,1) + 2 * lambda_ * w # (d,1)
    return output # (d,1)

### Implementing the gradient methods

In [ ]:
def solver(x, y, w, alpha, num_iters, lambda_, epsilon, optimizer = "GD", mem=False, return_cost=False, verbose=True):
    if (optimizer == "GD"):
        cost_ = np.zeros(num_iters)
        for i in range(num_iters):
            g = function_gradient(x, y, w, lambda_) # compute the exact gradient wrt w, given x and y
            w = w - alpha * g # GD update
            
            if return_cost:
                cost_[i] = cost(X_test, Y_test, w, lambda_)
            
            if (i%10==0) and (mem) and ('resource' in sys.modules) and (verbose):
                usage=resource.getrusage(resource.RUSAGE_SELF)
                print("mem for GD (mb):", (usage[2]*resource.getpagesize())/1000000.0)
                        
            if (np.linalg.norm(g) <= epsilon) and (verbose):
                print("GD: Stop on condition fulfilled! Number of iterations: ", i+1, "/", num_iters)
                break
        if verbose:
            print("GD: Number of iterations: ", num_iters, "/", num_iters)
            print("GD: final norm: ", np.linalg.norm(g))
                
    elif (optimizer == "SGD"):
        cost_ = np.zeros(num_iters)
        N, d = x.shape
        assert y.shape == (N,1)
        for i in range(num_iters):
            i_ = int(N*np.random.rand(1))
            x_ = x[[i_], :]
            y_ = y[[i_], :]
            
            g = function_gradient(x_, y_, w, lambda_) # compute gradient
            w = w - alpha * g # GD update 
            
            if return_cost:
                cost_[i] = cost(X_test, Y_test, w, lambda_)
            
            if (i%10==0) and (mem) and ('resource' in sys.modules) and (verbose):
                usage=resource.getrusage(resource.RUSAGE_SELF)
                print("mem for SGD (mb):", (usage[2]*resource.getpagesize())/1000000.0)
                
    elif (optimizer == "SVRG"):
        N, d = x.shape
        assert y.shape == (N,1)
        
        T = 100 # epoch length
        K = math.floor(num_iters/T) # number of epochs, given number of iterations
        
        if K == 0:
            # only run for one epoch consisting of num_iters iterations
            K = 1
            T = num_iters
        
        cost_ = np.zeros(T*K)
        
        for k in range(K):
            yx = y * x # (N, d)
            yxw = yx @ w # (N, 1)
            
            assert yx.shape == (N, d)
            assert yxw.shape == (N, 1)
            
            # G_ is equal to using function_gradient, but we need G_vec, so we compute it manually here
            exp_vec = 1/(1 + np.exp(yxw)) # (N, 1)
            
            G_vec = (-yx * exp_vec).reshape(d, N) + 2*lambda_*w # (d, N)
                                    
            G_ = (1/N) * np.sum(G_vec, axis=1, keepdims=1) # (d, 1)
            
            if (np.linalg.norm(G_) <= epsilon) and (verbose): # check the average gradient
                print("SVRG: Stop on condition fulfilled! Number of iterations: ", k*t, "/", num_iters)
                break
            
            assert G_vec.shape == (d, N)
            assert G_.shape == (d, 1)
            
            for t in range(T):
                i_ = int(N*np.random.rand(1))
                x_ = x[[i_], :] # (1, d)
                y_ = y[[i_], :] # (1, 1)
                g = function_gradient(x_, y_, w, lambda_) # (d, 1)
                assert g.shape == (d,1)
                assert G_vec[:, [i_]].shape == (d,1)
                assert G_.shape == (d,1)
                w = w - alpha*(g - G_vec[:, [i_]] + G_)
                
                if return_cost:
                    cost_[t + k*T] = cost(X_test, Y_test, w, lambda_)
                    
                if (t%10==0) and (mem) and ('resource' in sys.modules) and (verbose):
                    usage=resource.getrusage(resource.RUSAGE_SELF)
                    print("mem for SVRG (mb):", (usage[2]*resource.getpagesize())/1000000.0)
                    
        if (K > 0) and (verbose):
            print("SVRG: final norm: ", np.linalg.norm(g))
            
    elif (optimizer == "SAG"):
        N, d = x.shape
        assert y.shape == (N,1)
        
        # compute initial gradient estimates for all datapoints
        yx = y * x # (N, d)
        yxw = yx @ w # (N, 1)
        exp_vec = 1/(1 + np.exp(yxw)) # (N, 1)
#         G_vec = (-yx * exp_vec).reshape(d, N) + 2 * lambda_ * w # (d, N)
        G_vec = np.zeros((d, N))
        cost_ = np.zeros(num_iters)
        for k in range(num_iters):
            #set_trace()
            i_ = int(N*np.random.rand(1))
            x_ = x[[i_], :] # (1, d)
            y_ = y[[i_], :] # (1, 1)
            g = function_gradient(x_, y_, w, lambda_) # (d, 1)
            G_vec[:, [i_]] = g # (d, N)
            w = w - (alpha/N) * np.sum(G_vec, axis=1, keepdims=1) # (d, 1)
            if return_cost:
                cost_[k] = cost(X_test, Y_test, w, lambda_)
                
            if (np.linalg.norm(g) <= epsilon) and (verbose):
                    print("Stop on condition fulfilled! Number of iterations: ", k, "/", num_iters)
                    break
                    
            if (t%10==0) and (mem) and ('resource' in sys.modules) and (verbose):
                    usage=resource.getrusage(resource.RUSAGE_SELF)
                    print("mem for SAG (mb):", (usage[2]*resource.getpagesize())/1000000.0)
        if verbose:
            print("final norm: ", np.linalg.norm(g))
    if return_cost:
        return w, cost_
    else:
        return w

### Setting the constant stepsize
In these implementations we are using constant stepsize.

We set regularization $\lambda$ and gradient stop condition $\varepsilon$:

In [ ]:
## Define solvers: GD, SGD, SVRG and SAG. 
# Setting the values here:

lambda_ = 10 # change the value 1e-6 is good for full matrices
epsilon = 0.000001 # change the value

Load ``Y_train`` and ``X_train`` into ``y`` and ``x``.
Initial guess ``w``:

In [ ]:
y = Y_train
x = X_train
print("x: ", x.shape)
print("y: ", y.shape)
N, d = x.shape
w = np.random.rand(d,1)*0.1  # Initialization of w

In [ ]:
# smoothness and convexity
L = find_smoothness(x, y, lambda_)
print("L: ", L)
scaling = 2
alpha = 1/(scaling*L)
alpha_string = "alpha = 1/({0}*L)".format(scaling) + ": "
print(alpha_string, alpha)

In [ ]:
#-------------------- GD Solver -----------------------
print(alpha)
num_iters = 10000 # change the value
start = time.time()
gd = solver(x, y, w, alpha, num_iters, lambda_, epsilon, optimizer = "GD", mem=False)
end = time.time()
print("Weights of GD after convergence: \n", gd.flatten())
cost_value = cost(X_test, Y_test, gd, lambda_) 
print("Cost of GD after convergence: ", cost_value)

print("Training time for GD: ", end-start)

In [ ]:
#-------------------- SGD Solver -----------------------
num_iters = 10 # change the value
start = time.time()
sgd = solver(x, y, w, alpha, num_iters, lambda_, epsilon, optimizer = "SGD", mem=False)
end = time.time()
print("Weights of SGD after convergence: \n", sgd.flatten())

cost_value = cost(X_test, Y_test, sgd, lambda_)  # Calculate the cost value
print("Cost of SGD after convergence: ", cost_value)

print("Training time for SGD: ", end-start)

In [ ]:
#-------------------- SVRG Solver -----------------------
num_iters = 10000 # change the value
start = time.time()
svrg = solver(x, y, w, alpha, num_iters, lambda_, epsilon, optimizer="SVRG", mem=False)
end = time.time()
print("\nWeights of SVRG after convergence: \n", svrg.flatten())

cost_value = cost(X_test, Y_test, svrg, lambda_)
print("Cost of SVRG after convergence: ", cost_value)
print("Training time for SVRG: ", end-start)

In [ ]:
#-------------------- SAG Solver -----------------------
num_iters = 10000 # change the value
start = time.time()
sag = solver(x, y, w, alpha, num_iters, lambda_, epsilon, optimizer="SAG", mem=False)
end = time.time()
print("Weights of SAG after convergence: \n", sag.flatten())

cost_value = cost(X_test, Y_test, sag, lambda_)
print("Cost of SAG after convergence: ", cost_value)
print("Training time for SAG: ", end-start)

In [ ]:
## Executing the iterations and plot the cost function here:
I_max = 100
ti= np.zeros((I_max,4))
cost_= np.zeros((I_max,4))
w = np.random.rand(d,1)*0.01
for i in range(I_max):
    print("......",i,".......")
    #--------------GD-------------------
    start = time.time()
    gd = solver(x, y, w, alpha, i+1, lambda_, epsilon, optimizer="GD", mem=False)
    end = time.time()

    cost_[i,0] = cost(X_test, Y_test, gd, lambda_)

    ti[i,0] = end-start

    #---------------SGD------------------
    start = time.time()
    sgd = solver(x, y, w, alpha, i+1, lambda_, epsilon, optimizer="SGD", mem=False)
    end = time.time()

    cost_[i,1] = cost(X_test, Y_test, sgd, lambda_)

    ti[i,1] = end-start
    
    #---------------SVRG----------------
    start = time.time()
    svrg = solver(x, y, w, alpha, i+1, lambda_, epsilon, optimizer="SVRG", mem=False)
    end = time.time()

    cost_[i,2] = cost(X_test, Y_test, svrg, lambda_)

    ti[i,2] = end-start
    
    #---------------SAG------------------
    start = time.time()
    sag = solver(x, y, w, alpha, i+1, lambda_, epsilon, optimizer="SAG", mem=False)
    end = time.time()

    cost_[i,3] = cost(X_test, Y_test, sag, lambda_)

    ti[i,3] = end-start
    
    #------------------------------------
    
    ## Pl the results:
    

l0 = plt.plot(cost_[:,0],color="C3")
l1 = plt.plot(cost_[:,1],color="C2")
l2 = plt.plot(cost_[:,2],color="C1")
l3 = plt.plot(cost_[:,3],color="C0")
# complete other plots here: 


plt.xlabel("Number of Iteration")
plt.ylabel("Cost")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])
plt.show()

l0 = plt.plot(ti[:,0],color="C3")
l1 = plt.plot(ti[:,1],color="C2")
l2 = plt.plot(ti[:,2],color="C1")
l3 = plt.plot(ti[:,3],color="C0")
# complete other plots here:

plt.xlabel("Number of Iteration")
plt.ylabel("Time (sec)")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])
plt.show()

In [ ]:
## Executing the iterations and plot the cost function here:
print(alpha_string, alpha)
n_sims = 10
I_max = 10000
ti= np.zeros((I_max,4))
cost_= np.zeros((I_max,4))
for n in range(n_sims):
    w = np.random.rand(d,1)*0.01
    
    print("......",n+1,".......")
    #--------------GD-------------------
    gd, gd_cost  = solver(x, y, w, alpha, I_max, lambda_, epsilon, optimizer="GD", mem=False, return_cost=True, verbose=False)

    cost_[:,0] += gd_cost/n_sims

    #---------------SGD------------------
    sgd, sgd_cost = solver(x, y, w, alpha, I_max, lambda_, epsilon, optimizer="SGD", mem=False, return_cost=True, verbose=False)

    cost_[:,1] += sgd_cost/n_sims
    
    #---------------SVRG----------------
    svrg, svrg_cost = solver(x, y, w, alpha, I_max, lambda_, epsilon, optimizer="SVRG", mem=False, return_cost=True, verbose=False)

    cost_[:,2] += svrg_cost/n_sims
    
    #---------------SAG------------------
    sag, sag_cost = solver(x, y, w, alpha, I_max, lambda_, epsilon, optimizer="SAG", mem=False, return_cost=True, verbose=False)
    
    cost_[:,3] += sag_cost/n_sims
    
    #------------------------------------
## PLOTs
l0 = plt.plot(cost_[:,0],color="C3")
l1 = plt.plot(cost_[:,1],color="C2")
l2 = plt.plot(cost_[:,2],color="C1")
l3 = plt.plot(cost_[:,3],color="C0")
# complete other plots here: 

plt.xlabel("Number of Iteration")
plt.ylabel("Cost")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])
plt.show()

In [ ]:
## PLOTs
l0 = plt.plot(cost_[:,0],color="C3")
l1 = plt.plot(cost_[:,1],color="C2")
l2 = plt.plot(cost_[:,2],color="C1")
l3 = plt.plot(cost_[:,3],color="C0")
# complete other plots here: 

plt.xlabel("Number of Iteration")
plt.ylabel("Cost")
plt.legend(['GD', 'SGD', 'SVRG', 'SAG'])

plt.ylim(0.25, .35)
plt.xlim(15000,20000)

In [ ]:
## Tunning the hyper-paramter here:

In [ ]:
## Comparing different optimizers here: 